In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
fp = open('data/raw.txt', 'r', encoding='utf-8', errors='ignore')
texts = fp.readlines()

In [ ]:
texts[:5]

In [ ]:
def clean_line(line):
    pattern = '\[(\d{2}\/\d{2}\/\d{4}), (\d{2}\:\d{2}\:\d{2})\]\s(?:([^\:]*)\:(.*)|(.*))'
    
    line = re.sub('\u200e', '', line)
    
    m = re.match(pattern, line)
    
    if m is None:
        return None, None, None, None
    
    date = m.group(1)
    time = m.group(2)
    
    event = False
    sender = None
    msg = None
    
    
    if m.group(3):
        sender = m.group(3)
        msg = m.group(4).strip()
    else:
        event = True
    
    dd = datetime.strptime(str(date) + " " + str(time), '%d/%m/%Y %H:%M:%S')
    
    return (dd, event, sender, msg)

In [ ]:
df = pd.DataFrame(data=list(map(clean_line, texts)), columns=['date', 'event', 'sender', 'msg'])

In [ ]:
df

In [ ]:
df = df.dropna(thresh=3, axis=0)

In [ ]:
df = df.drop(['event'], axis=1)

In [ ]:
df_g = df.set_index('date').groupby(pd.Grouper(freq='D')).count()

In [ ]:
df_g = df_g[df_g['sender'] != 0]

In [ ]:
df_g

In [ ]:
fig = plt.figure(figsize=(12, 8))
plt.bar(df_g.index, df_g['sender'])
plt.xticks(rotation=90)

In [ ]:
df_g.to_csv('data/processed_nacut.csv')